## 0. Import dependencies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sys
sys.path.insert(0, '../utils')
from load_data import ames_housing
from preprocess_data import cat2int

import functools
import tensorflow as tf
#from tensorflow.python.feature_column.feature_column import _LazyBuilder
tf.__version__

'1.9.0'

## 1. Load data

In [2]:
dirfolder = '../data/Ames-Housing'
numvalid = 100
train, valid, testdf = ames_housing(dirfolder, numvalid)
traindf, trainy = train
validdf, validy = valid

../utils/load_data.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  testdf_new['Id'] = test_ID


## 2. Preprocess input data

In [3]:
traindata = traindf.values
validdata = validdf.values
testdata = testdf.values

# normalization
max_val = np.max(traindata, axis=0)
min_val = np.min(traindata, axis=0)
for i in range(traindata.shape[1]):
    if max_val[i] != 0:
        traindata[:, i] = (traindata[:, i]-min_val[i])/(max_val[i]-min_val[i])
        validdata[:, i] = (validdata[:, i]-min_val[i])/(max_val[i]-min_val[i])
        testdata[:, i] = (testdata[:, i]-min_val[i])/(max_val[i]-min_val[i])

## 3. Create dataflow graphs

In [4]:
# hyperparameters
lr_decay = 1.0
lr_0 = 1.0
batch_size = 32
iterations = 500000
monitor = 10000
perf_reduce = 0.3
keep_prob_input_0 = 0.8
keep_prob_a_0 = 0.5

# network size
num_train, num_in = traindata.shape
num_hidden_1 = 100
num_hidden_2 = 50
num_hidden_3 = 25
num_out = 1

# create network graph
## input and hidden layer 1
x = tf.placeholder(tf.float32, [None, num_in])
keep_prob_input = tf.placeholder(tf.float32)
keep_prob_a = tf.placeholder(tf.float32)
x_drop = tf.nn.dropout(x, keep_prob_input)
W_1 = tf.Variable(tf.random_normal([num_in, num_hidden_1], stddev=0.1))
b_1 = tf.Variable(tf.random_normal([num_hidden_1], stddev=0.1))
y_1 = tf.matmul(x_drop, W_1) + b_1
a_1 = tf.nn.selu(y_1)
bn_a_1 = tf.layers.batch_normalization(a_1)
a_1_drop = tf.nn.dropout(bn_a_1, keep_prob_a)

## hidden layer 1 and 2
W_2 = tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], stddev=0.1))
b_2 = tf.Variable(tf.random_normal([num_hidden_2], stddev=0.1))
y_2 = tf.matmul(a_1_drop, W_2) + b_2
a_2 = tf.nn.selu(y_2)
bn_a_2 = tf.layers.batch_normalization(a_2)
a_2_drop = tf.nn.dropout(bn_a_2, keep_prob_a)

## hidden layer 2 and 3
W_3 = tf.Variable(tf.random_normal([num_hidden_2, num_hidden_3], stddev=0.1))
b_3 = tf.Variable(tf.random_normal([num_hidden_3], stddev=0.1))
y_3 = tf.matmul(a_2_drop, W_3) + b_3
a_3 = tf.nn.selu(y_3)
bn_a_3 = tf.layers.batch_normalization(a_3)
a_3_drop = tf.nn.dropout(bn_a_3, keep_prob_a)

## hidden layer 3 and output 
W_4 = tf.Variable(tf.random_normal([num_hidden_3, num_out], stddev=0.1))
b_4 = tf.Variable(tf.random_normal([num_out], stddev=0.1))
y_out = tf.matmul(a_3_drop, W_4) + b_4

# create training graph
learning_rate = tf.placeholder(tf.float32)
y_ = tf.placeholder(tf.float32, [None, num_out])
cost = tf.losses.mean_squared_error(y_, y_out)
train = tf.train.AdadeltaOptimizer(learning_rate).minimize(cost)

## 4. Training

In [ ]:
# initialize
session = tf.Session()
session.run(tf.global_variables_initializer())

# train
start = 0
last_perf_train = np.inf
last_perf_valid = np.inf
best_perf_valid = np.inf
saver = tf.train.Saver()
for i in range(iterations):
    session.run(train, feed_dict={x: traindata[start:start+batch_size, :], 
                                  y_: trainy[start:start+batch_size],
                                  keep_prob_input: keep_prob_input_0,
                                  keep_prob_a: keep_prob_a_0, 
                                  learning_rate: lr_0})
    start += batch_size
    if start > num_train-batch_size: 
        start = 0
    if i % monitor == 0:
        perf_train = session.run(cost, feed_dict={x: traindata, 
                                                  y_: trainy,
                                                  keep_prob_input: 1.0,
                                                  keep_prob_a: 1.0})
        perf_valid = session.run(cost, feed_dict={x: validdata, 
                                                  y_: validy,
                                                  keep_prob_input: 1.0,
                                                  keep_prob_a: 1.0})
        if (best_perf_valid != np.inf) and ((perf_valid-best_perf_valid)/best_perf_valid > perf_reduce):
            print("Difference between two consercutive costs is too small.")
            break
        last_perf_train = perf_train
        last_perf_valid = perf_valid
        if perf_valid < best_perf_valid:
            save_path = saver.save(session, "model/dnnreg_model.ckpt")
            print("Better model has been saved.")
            best_perf_valid = perf_valid
        lr_0 = lr_0*lr_decay
        print("{}: {}, {}".format(i, perf_train, perf_valid))
session.close()

Better model has been saved.
0: 137.41915893554688, 136.74462890625
Better model has been saved.
10000: 0.1967027485370636, 0.18853135406970978
Better model has been saved.
20000: 0.0823640450835228, 0.07588567584753036
Better model has been saved.
30000: 0.07029766589403152, 0.06608142703771591
Better model has been saved.
40000: 0.038593754172325134, 0.03779486194252968
Better model has been saved.
50000: 0.01817265711724758, 0.021625518798828125
60000: 0.019349901005625725, 0.02414541319012642
Better model has been saved.
70000: 0.014460388571023941, 0.020654449239373207
80000: 0.013785775750875473, 0.02085169404745102
90000: 0.013292822055518627, 0.02122700773179531
100000: 0.013471725396811962, 0.02119981311261654
110000: 0.013298382051289082, 0.021411821246147156
120000: 0.01412172894924879, 0.022094642743468285
130000: 0.013055992312729359, 0.020729275420308113
Better model has been saved.
140000: 0.011763963848352432, 0.019622011110186577
150000: 0.012140484526753426, 0.0209642

## 5. Predict

In [ ]:
session = tf.Session()
saver.restore(session, 'model/dnnreg_model.ckpt')
perf_train = session.run(cost, feed_dict={x: traindata, 
                                          y_: trainy,
                                          keep_prob_input: 1.0,
                                          keep_prob_a: 1.0})
perf_valid = session.run(cost, feed_dict={x: validdata, 
                                          y_: validy,
                                          keep_prob_input: 1.0,
                                          keep_prob_a: 1.0})
print(perf_train, perf_valid)
predtrainy = session.run(y_out, feed_dict={x: traindata, 
                                           keep_prob_input: 1.0,
                                           keep_prob_a: 1.0})
predvalidy = session.run(y_out, feed_dict={x: validdata, 
                                           keep_prob_input: 1.0,
                                           keep_prob_a: 1.0})
predtesty = session.run(y_out, feed_dict={x: testdata, 
                                           keep_prob_input: 1.0,
                                           keep_prob_a: 1.0})
session.close()

## 5. Plot results

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.expm1(trainy), label='True')
ax.plot(np.expm1(predtrainy), label='Predict')
ax.set_xlabel('Train Id')
ax.set_ylabel('Sale Price')
ax.set_title('Training result');

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.expm1(validy), label='True')
ax.plot(np.expm1(predvalidy), label='Predict')
ax.set_xlabel('Valid Id')
ax.set_ylabel('Sale Price')
ax.set_title('Validation result');

## 6. Save testing result for submission

In [72]:
sub_dict = {'Id': testdf.index.tolist(), 'SalePrice': np.expm1(predtesty).flatten()}
dnnreg_submission = pd.DataFrame(sub_dict)
dnnreg_submission.set_index('Id', inplace=True)
dnnreg_submission.to_csv('dnnreg_submission')